In [5]:
import spacy
from spacy.tokens import DocBin
import pandas as pd
import re

# funcs

In [16]:
import re

def create_entity_spans(data, tag_list):
    training_data = []

    for idx, row in data.iterrows():
        address = row['address']
        entities = []

        # Город
        city_match = re.match(r'([^,]+?) г,?', address)
        if city_match:
            start, end = city_match.span(1)
            entities.append((start, end, 'Город'))

        # Улица
        street_match = re.search(r'ул ([^,]+)', address)
        if street_match:
            start, end = street_match.span(1)
            entities.append((start, end, 'Улица'))

        # Дом
        building_match = re.search(r'(\d+)\s?', address)
        if building_match:
            start, end = building_match.span(1)
            entities.append((start, end, 'Дом'))

        # Корпус
        corpus_match = re.search(r'к\.?\s?(\d+)', address)
        if corpus_match:
            start, end = corpus_match.span(1)
            entities.append((start, end, 'Корпус'))

        # Литера
        litera_match = re.search(r'литера\s?([А-ЯA-Z])', address, re.IGNORECASE)
        if litera_match:
            start, end = litera_match.span(1)
            entities.append((start, end, 'Литера'))

        # Квартира
        flat_match = re.search(r'кв\.?\s?(\d+)', address)
        if flat_match:
            start, end = flat_match.span(1)
            entities.append((start, end, 'Квартира'))

        training_data.append((address, entities))

    return pd.DataFrame(training_data, columns=["address", "entities"])


from spacy.tokens import DocBin
from spacy.training import Example

def get_doc_bin(training_data, nlp):
    doc_bin = DocBin()  # контейнер для сериализации

    for text, entities in training_data:
        doc = nlp.make_doc(text)
        ents = []

        for start, end, label in entities:
            span = doc.char_span(start, end, label=label)
            if span is None:
                print(f"⚠️ Пропущен span: ({start}, {end}, '{label}') в тексте: '{text[start:end]}'")
                continue
            ents.append(span)

        doc.ents = ents
        doc_bin.add(doc)

    return doc_bin



# import

In [8]:
import pandas as pd
from google.colab import drive

drive.mount('mydrive')

data0=pd.read_csv("/content/mydrive/MyDrive/life/EXplore/мероприятия/Марафон \"Цифровой Петербург\"/FuzzySearch/данные/fias_dict.csv")
data = data0[:1000]
datavl = data0[1000:2000]
data.head()

Drive already mounted at mydrive; to attempt to forcibly remount, call drive.mount("mydrive", force_remount=True).


,id,address
0,4812,"Санкт-Петербург г, ул Полоцкая, 15 к. 2 литера..."
1,4813,"Санкт-Петербург г, ул Полоцкая, 15 к. 2 литера..."
2,4814,"Санкт-Петербург г, ул Полоцкая, 15 к. 2 литера..."
3,4815,"Санкт-Петербург г, ул Полоцкая, 15 к. 2 литера..."
4,4816,"Санкт-Петербург г, ул Полоцкая, 15 к. 2 литера..."


#data

In [9]:
tag_list=["Улица","Дом","корпус","Литера","Квартира"]

In [6]:
#nlp = spacy.blank("ru")

In [10]:
nlp = spacy.load('ru_core_news_lg')

In [1]:
from spacy.cli import download

download("ru_core_news_lg")


import spacy

nlp = spacy.load("ru_core_news_lg")


✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [11]:
nlp


In [12]:
df_entity_spans= create_entity_spans(data.astype(str),tag_list)
df_entity_spans.head()

,address,entities
0,"Санкт-Петербург г, ул Полоцкая, 15 к. 2 литера...","[(0, 15, Город), (22, 30, Улица), (32, 34, Дом..."
1,"Санкт-Петербург г, ул Полоцкая, 15 к. 2 литера...","[(0, 15, Город), (22, 30, Улица), (32, 34, Дом..."
2,"Санкт-Петербург г, ул Полоцкая, 15 к. 2 литера...","[(0, 15, Город), (22, 30, Улица), (32, 34, Дом..."
3,"Санкт-Петербург г, ул Полоцкая, 15 к. 2 литера...","[(0, 15, Город), (22, 30, Улица), (32, 34, Дом..."
4,"Санкт-Петербург г, ул Полоцкая, 15 к. 2 литера...","[(0, 15, Город), (22, 30, Улица), (32, 34, Дом..."


In [13]:
training_data= df_entity_spans.values.tolist()

In [14]:
training_data

[['Санкт-Петербург г, ул Полоцкая, 15 к. 2 литера А, кв. 87',
  [(0, 15, 'Город'),
   (22, 30, 'Улица'),
   (32, 34, 'Дом'),
   (38, 39, 'Корпус'),
   (47, 48, 'Литера'),
   (54, 56, 'Квартира')]],
 ['Санкт-Петербург г, ул Полоцкая, 15 к. 2 литера А, кв. 13',
  [(0, 15, 'Город'),
   (22, 30, 'Улица'),
   (32, 34, 'Дом'),
   (38, 39, 'Корпус'),
   (47, 48, 'Литера'),
   (54, 56, 'Квартира')]],
 ['Санкт-Петербург г, ул Полоцкая, 15 к. 2 литера А, кв. 128',
  [(0, 15, 'Город'),
   (22, 30, 'Улица'),
   (32, 34, 'Дом'),
   (38, 39, 'Корпус'),
   (47, 48, 'Литера'),
   (54, 57, 'Квартира')]],
 ['Санкт-Петербург г, ул Полоцкая, 15 к. 2 литера А, кв. 122',
  [(0, 15, 'Город'),
   (22, 30, 'Улица'),
   (32, 34, 'Дом'),
   (38, 39, 'Корпус'),
   (47, 48, 'Литера'),
   (54, 57, 'Квартира')]],
 ['Санкт-Петербург г, ул Полоцкая, 15 к. 2 литера А, кв. 81',
  [(0, 15, 'Город'),
   (22, 30, 'Улица'),
   (32, 34, 'Дом'),
   (38, 39, 'Корпус'),
   (47, 48, 'Литера'),
   (54, 56, 'Квартира')]],
 ['Санкт

In [17]:
doc_bin_train= get_doc_bin(training_data,nlp)
list(doc_bin_train.get_docs(nlp.vocab))

⚠️ Пропущен span: (23, 24, 'Дом') в тексте: '7'
⚠️ Пропущен span: (23, 24, 'Дом') в тексте: '9'
⚠️ Пропущен span: (23, 24, 'Дом') в тексте: '9'
⚠️ Пропущен span: (23, 24, 'Дом') в тексте: '9'
⚠️ Пропущен span: (23, 24, 'Дом') в тексте: '9'
⚠️ Пропущен span: (23, 24, 'Дом') в тексте: '9'
⚠️ Пропущен span: (23, 24, 'Дом') в тексте: '9'
⚠️ Пропущен span: (35, 36, 'Дом') в тексте: '7'
⚠️ Пропущен span: (44, 45, 'Литера') в тексте: 'А'
⚠️ Пропущен span: (35, 36, 'Дом') в тексте: '2'
⚠️ Пропущен span: (35, 36, 'Дом') в тексте: '2'
⚠️ Пропущен span: (44, 45, 'Литера') в тексте: 'А'
⚠️ Пропущен span: (52, 53, 'Квартира') в тексте: '1'
⚠️ Пропущен span: (35, 37, 'Дом') в тексте: '12'
⚠️ Пропущен span: (35, 37, 'Дом') в тексте: '12'
⚠️ Пропущен span: (35, 37, 'Дом') в тексте: '12'
⚠️ Пропущен span: (35, 36, 'Дом') в тексте: '3'
⚠️ Пропущен span: (44, 45, 'Литера') в тексте: 'А'
⚠️ Пропущен span: (44, 45, 'Литера') в тексте: 'А'
⚠️ Пропущен span: (44, 45, 'Литера') в тексте: 'А'
⚠️ Пропущен span:

[Санкт-Петербург г, ул Полоцкая, 15 к. 2 литера А, кв. 87,
 Санкт-Петербург г, ул Полоцкая, 15 к. 2 литера А, кв. 13,
 Санкт-Петербург г, ул Полоцкая, 15 к. 2 литера А, кв. 128,
 Санкт-Петербург г, ул Полоцкая, 15 к. 2 литера А, кв. 122,
 Санкт-Петербург г, ул Полоцкая, 15 к. 2 литера А, кв. 81,
 Санкт-Петербург г, ул Полоцкая, 15 к. 2 литера А, кв. 135,
 Санкт-Петербург г, ул Полоцкая, 15 к. 2 литера А, кв. 85,
 Санкт-Петербург г, ул Полоцкая, 15 к. 2 литера А, кв. 20,
 Санкт-Петербург г, ул Полоцкая, 15 к. 2 литера А, кв. 53,
 Санкт-Петербург г, ул Полоцкая, 15 к. 2 литера А, кв. 134,
 Санкт-Петербург г, ул Полоцкая, 15 к. 2 литера А, кв. 113,
 Санкт-Петербург г, ул Полоцкая, 15 к. 2 литера А, кв. 45,
 Санкт-Петербург г, ул Полоцкая, 15 к. 2 литера А, кв. 111,
 Санкт-Петербург г, ул Полоцкая, 15 к. 2 литера А, кв. 124,
 Санкт-Петербург г, ул Полоцкая, 15 к. 2 литера А, кв. 19,
 Санкт-Петербург г, ул Полоцкая, 15 к. 2 литера А, кв. 116,
 Санкт-Петербург г, ул Полоцкая, 15 к. 2 литера 

In [18]:
doc_bin_train.to_disk("./train.spacy")

## val

In [19]:
df_entity_spans= create_entity_spans(datavl.astype(str),tag_list)
validation_data= df_entity_spans.values.tolist()
doc_bin_vl= get_doc_bin(validation_data,nlp)
doc_bin_vl.to_disk("./test.spacy")
list(doc_bin_vl.get_docs(nlp.vocab))

⚠️ Пропущен span: (23, 24, 'Дом') в тексте: '9'
⚠️ Пропущен span: (33, 35, 'Дом') в тексте: '11'
⚠️ Пропущен span: (33, 35, 'Дом') в тексте: '11'
⚠️ Пропущен span: (33, 35, 'Дом') в тексте: '11'
⚠️ Пропущен span: (33, 35, 'Дом') в тексте: '11'
⚠️ Пропущен span: (33, 35, 'Дом') в тексте: '11'
⚠️ Пропущен span: (33, 35, 'Дом') в тексте: '11'
⚠️ Пропущен span: (33, 35, 'Дом') в тексте: '11'
⚠️ Пропущен span: (33, 35, 'Дом') в тексте: '11'
⚠️ Пропущен span: (33, 35, 'Дом') в тексте: '11'
⚠️ Пропущен span: (33, 35, 'Дом') в тексте: '11'
⚠️ Пропущен span: (33, 35, 'Дом') в тексте: '11'
⚠️ Пропущен span: (33, 35, 'Дом') в тексте: '11'
⚠️ Пропущен span: (33, 35, 'Дом') в тексте: '11'
⚠️ Пропущен span: (33, 35, 'Дом') в тексте: '11'
⚠️ Пропущен span: (33, 35, 'Дом') в тексте: '11'
⚠️ Пропущен span: (33, 35, 'Дом') в тексте: '11'
⚠️ Пропущен span: (33, 35, 'Дом') в тексте: '11'
⚠️ Пропущен span: (33, 35, 'Дом') в тексте: '11'
⚠️ Пропущен span: (33, 35, 'Дом') в тексте: '11'
⚠️ Пропущен span: (33

[Санкт-Петербург г, пер Спасский, 6-8 литера А , помещ. 8-Н,
 Санкт-Петербург г, пер Спасский, 6-8 литера А , кв. 144,
 Санкт-Петербург г, пер Спасский, 6-8 литера А , кв. 24,
 Санкт-Петербург г, пер Спасский, 6-8 литера А , кв. 53,
 Санкт-Петербург г, пер Спасский, 6-8 литера А , кв. 9,
 Санкт-Петербург г, пер Спасский, 6-8 литера А , кв. 152,
 Санкт-Петербург г, пер Спасский, 6-8 литера А , кв. 45,
 Санкт-Петербург г, пер Спасский, 6-8 литера А , помещ. 4ЛК,
 Санкт-Петербург г, пер Спасский, 6-8 литера А , кв. 18,
 Санкт-Петербург г, пер Спасский, 6-8 литера А , помещ. 3ЛК,
 Санкт-Петербург г, пер Спасский, 6-8 литера А , помещ. 30-Н,
 Санкт-Петербург г, пер Спасский, 6-8 литера А , кв. 71,
 Санкт-Петербург г, пер Спасский, 6-8 литера А , кв. 2,
 Санкт-Петербург г, пер Спасский, 6-8 литера А , кв. 32,
 Санкт-Петербург г, пер Спасский, 6-8 литера А , помещ. 9-Н,
 Санкт-Петербург г, пер Спасский, 6-8 литера А , кв. 154,
 Санкт-Петербург г, пер Спасский, 6-8 литера А , кв. 44,
 Санкт-Пе

# train

In [20]:
!pip install spacy-transformers  # если хочешь использовать трансформеры

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.2/756.2 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.0/314.0 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 64.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 62.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [25]:
!python -m spacy init config config.cfg --lang ru --pipeline ner,transformer --optimize accuracy
#!python -m spacy init config config.cfg --lang en --pipeline ner

ℹ Generated config template specific for your use case
- Language: ru
- Pipeline: ner
- Optimize for: accuracy
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [26]:
!python -m spacy train config.cfg --output ./output --paths.train ./train.spacy --paths.dev ./test.spacy


✔ Created output directory: output
ℹ Saving to output directory: output
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     72.73   22.70   18.88   28.46    0.23
  1     200          4.85    768.89   94.48   89.53  100.00    0.94
  3     400          0.29      2.59   94.81   90.13  100.00    0.95
  5     600          0.00      0.00   94.81   90.13  100.00    0.95
  8     800          0.00      0.00   97.57   95.26  100.00    0.98
 11    1000          0.00      0.00   97.57   95.26  100.00    0.98
 15    1200          0.00      0.00   95.44   91.27  100.00    0.95
 20    1400          0.00      0.00   95.44   91.27  100.00    0

In [36]:
import spacy

nlp = spacy.load("./output/model-best")

address_list=["Санкт ул Маяковского 14 к 2 литера В кв 13",
              'Маяковского']

for address in address_list:
    doc=nlp(address)
    ent_list=[(ent.text, ent.label_) for ent in doc.ents]
    print("Address string -> "+address)
    print("Parsed address -> "+str(ent_list))
    print("******")

Address string -> Санкт ул Маяковского 14 к 2 литера В кв 13
Parsed address -> [('Санкт ул Маяковского', 'Город'), ('14', 'Дом'), ('2', 'Корпус'), ('В', 'Литера'), ('13', 'Квартира')]
******
Address string -> Маяковского
Parsed address -> [('Маяковского', 'Литера')]
******
